In [5]:
import mysql.connector as mysql
import pandas as pd
import numpy
import sys

# Read the Excel file
try:
    data = pd.read_excel('/Users/maniraj/Downloads/GSTR2.xls')
    print("Excel file read successfully. Preview of the data:")
    print(data.head())
    print("\nColumns in the Excel file:")
    print(data.columns.tolist())
    print("\nData types:")
    print(data.dtypes)
except Exception as e:
    print(f"Error reading Excel file: {e}")
    sys.exit(1)

# Database connection parameters
DB_CONFIG = {
    'user': 'myuser',
    'password': 'mypassword',
    'host': 'localhost',
    'port': 3306,
    'database': 'mydatabase'
}

# Connect to the database
try:
    conn = mysql.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("\nConnected to MySQL database successfully")
except Exception as e:
    print(f"Error connecting to MySQL database: {e}")
    sys.exit(1)

# Create table based on DataFrame structure
try:
    table_name = "gstr4_data_1"
    
    def map_dtype(dtype, col):
        # Force VOUCHERNUMBER to VARCHAR(32)
        if str(col).lower() == 'vouchernumber':
            return 'VARCHAR(32)'
        if pd.api.types.is_integer_dtype(dtype):
            return 'INT'
        elif pd.api.types.is_float_dtype(dtype):
            return 'FLOAT'
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            return 'DATETIME'
        else:
            return 'VARCHAR(255)'

    # Clean column names for SQL
    def clean_col(col):
        return ''.join(e if e.isalnum() else '_' for e in str(col))
    
    columns = []
    for col, dtype in zip(data.columns, data.dtypes):
        col_type = map_dtype(dtype, col)
        clean_column = clean_col(col)
        columns.append(f"{clean_column} {col_type}")

    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    create_table_sql += ",\n".join(columns)
    create_table_sql += "\n);"
    
    print("\nExecuting SQL to create table:")
    print(create_table_sql)
    
    cursor.execute(create_table_sql)
    print(f"\nTable '{table_name}' created successfully")
    
except Exception as e:
    print(f"Error creating table: {e}")
    conn.close()
    sys.exit(1)

# Import data into the table
try:
    # Convert VOUCHERNUMBER to string in pandas DataFrame
    if 'VOUCHERNUMBER' in data.columns:
        data['VOUCHERNUMBER'] = data['VOUCHERNUMBER'].astype(str)
    
    values = []
    for _, row in data.iterrows():
        cleaned_row = [None if pd.isna(val) else val for val in row]
        values.append(cleaned_row)
    
    placeholders = ', '.join(['%s'] * len(data.columns))
    clean_columns = [clean_col(col) for col in data.columns]
    columns_str = ', '.join(clean_columns)
    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"
    
    cursor.executemany(insert_query, values)
    conn.commit()
    
    print(f"\nSuccessfully imported {len(values)} rows into table '{table_name}'")
    
except Exception as e:
    print(f"Error importing data: {e}")
    conn.rollback()
finally:
    cursor.close()
    conn.close()
    print("\nDatabase connection closed")


Excel file read successfully. Preview of the data:
  DOCUMENTTYPE   DOCUMENTNUMBER DOCUMENTDATE RETURNFILINGMONTH  PLACEOFSUPPLY  \
0      INVOICE               39   2025-04-04            APR-25            NaN   
1      INVOICE               40   2025-04-04            APR-25            NaN   
2      INVOICE               40   2025-04-04            APR-25            NaN   
3      INVOICE               40   2025-04-04            APR-25            NaN   
4      INVOICE  KABLG2410025109   2025-03-31            APR-25            NaN   

  ISBILLOFSUPPLY ISREVERSECHARGE  LINKEDADVANCEDOCUMENTNUMBER  \
0              N               N                          NaN   
1              N               N                          NaN   
2              N               N                          NaN   
3              N               N                          NaN   
4              N               N                          NaN   

   LINKEDADVANCEDOCUMENTDATE  LINKEDADVANCEADJUSTMENTAMOUNT  ... COMPAN

In [8]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 7.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install mysql

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 91 kB 8.2 MB/s eta 0:00:01
  distutils: /private/var/folders/72/c82whp490qzdyfd7bkg1b6qm0000gn/T/pip-build-env-90falraf/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/72/c82whp490qzdyfd7bkg1b6qm0000gn/T/pip-build-env-90falraf/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/72/c82whp490qzdyfd7bkg1b6qm0000gn/T/pip-build-env-90falraf/normal'
  distutils: /private/var/folders/72/c82whp490qzdyfd7bkg1b6qm0000gn/T/pip-build-env-90falraf/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/72/c82whp490qzdyfd7bkg1b6qm0000gn/T/pip-build-env-90falraf/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = F

In [24]:
import mysql.connector as mysql
import pandas as pd
import numpy as np
import sys
import os

# Read the Excel file
excel_file = '/Users/maniraj/Downloads/GSTR2.xls'
try:
    data = pd.read_excel(excel_file)
    print("Excel file read successfully. Preview of the data:")
    print(data.head())
    print("\nColumns in the Excel file:")
    print(data.columns.tolist())
    print("\nData types:")
    print(data.dtypes)
except Exception as e:
    print(f"Error reading Excel file: {e}")
    sys.exit(1)

# Database connection parameters
DB_CONFIG = {
    'user': 'myuser',
    'password': 'mypassword',
    'host': 'localhost',
    'port': 3306,
    'database': 'mydatabase'
}

# Generate table name from Excel filename
table_name = "gstr2_data_1"

# Map pandas dtypes to MySQL data types
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'VARCHAR(255)'

# Clean column names for SQL
def clean_col(col):
    return ''.join(e if e.isalnum() else '_' for e in str(col))

columns = []
for col, dtype in zip(data.columns, data.dtypes):
    columns.append(f"`{clean_col(col)}` {map_dtype(dtype)}")

create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` (\n"
create_table_sql += ",\n".join(columns)
create_table_sql += "\n);"

# Connect to the database and execute SQL
try:
    with mysql.connect(**DB_CONFIG) as conn:
        with conn.cursor() as cursor:
            print("\nConnected to MySQL database successfully")
            print("\nExecuting SQL to create table:")
            print(create_table_sql)
            cursor.execute(create_table_sql)
            print(f"\nTable '{table_name}' created successfully")
            
            # Prepare data for insertion
            values = []
            for _, row in data.iterrows():
                cleaned_row = [None if pd.isna(val) else val for val in row]
                values.append(cleaned_row)
            
            placeholders = ', '.join(['%s'] * len(data.columns))
            clean_columns = [f"`{clean_col(col)}`" for col in data.columns]
            columns_str = ', '.join(clean_columns)
            insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"
            
            cursor.executemany(insert_query, values)
            conn.commit()
            print(f"\nSuccessfully imported {len(values)} rows into table '{table_name}'")
except Exception as e:
    print(f"Database operation failed: {e}")
    sys.exit(1)

print("\nDatabase connection closed")


ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [23]:
pip install xlrd


4961.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip3 uninstall -y numpy pandas
rm -rf ~/Library/Python/3.9/lib/python/site-packages/numpy*
rm -rf ~/Library/Python/3.9/lib/python/site-packages/pandas*
pip3 install numpy==1.26.4 pandas --upgrade


ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [26]:
pip install pandas

5047.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [29]:
pip install numpy==1.26.4

5094.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [ ]:
python3 -m venv myenv
source myenv/bin/activate



SyntaxError: invalid syntax (3701276763.py, line 1)

In [32]:
pip install mysql-connector-python pandas numpy xlrd openpyxl

5190.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
